#Project Data Analysis
>By Gligore Emil

##Importing libraries
>I import Pandas for data manipulation, Numpy for calculations, LinearRegression from Sklearn in order to train a model, and the two Plotly libraries for plotting.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.graph_objs as go
from plotly.subplots import make_subplots

##Loading the datasets

In [ ]:
gdp_df = pd.read_csv("/content/gdpromania.csv")
inflation_df = pd.read_csv("/content/inflationromania.csv")
unemployment_df = pd.read_csv("/content/youthunemploymentromania.csv")
stock_market_df = pd.read_csv("/content/roxde.csv")

##Data preprocessing
>I make the date columns datetime and I also rename some of the columns since they had weird names.

In [ ]:
gdp_df['Date'] = pd.to_datetime(gdp_df['DATE'])
inflation_df['Date'] = pd.to_datetime(inflation_df['DATE'])
unemployment_df['Date'] = pd.to_datetime(unemployment_df['DATE'])
stock_market_df['Date'] = pd.to_datetime(stock_market_df['Date'])

In [ ]:
gdp_df.rename(columns={'CPMNACSCAB1GQRO': 'GDP'}, inplace=True)
inflation_df.rename(columns={'FPCPITOTLZGROU': 'Inflation'}, inplace=True)
unemployment_df.rename(columns={'SLUEM1524ZSROU': 'Youth Unemployment'}, inplace=True)
stock_market_df.rename(columns={'Close': 'Stock Market'}, inplace=True)

##Function to calculate year to year percentage change
>This function takes in a dataframe and the column we want to see the percetange change.
First line creates a Year column by extracting the year from the date column.
Second line creates a Percentange column by performing the pandas function pct_change on the column we want to analyze, and then multypling it by 100 to get the percetange.
Third line we group by year and then we make a mean of the percentage change for the whole year, and then we reset the index so the year column is not an index anymore.

In [ ]:
def calculate_percentage_change(df, column):
    df['Year'] = df['Date'].dt.year
    df['Percentage Change'] = df[column].pct_change() * 100
    return df.groupby('Year')['Percentage Change'].mean().reset_index()

##Calculate percentage changes
>Run the function on all of the datasets.

In [ ]:
gdp_percent_change = calculate_percentage_change(gdp_df, 'GDP')
unemployment_percent_change = calculate_percentage_change(unemployment_df, 'Youth Unemployment')
inflation_percent_change = calculate_percentage_change(inflation_df, 'Inflation')
stock_market_percent_change = calculate_percentage_change(stock_market_df, 'Stock Market')

##Function to predict future values using LinearRegression
>This function takes in a dataframe, the column we want to predict, and how many years to predict in the future.
The first line is the same as in the first function.
X is equal to the Year column without the rows with missing values.
Y is equal to the column we want to predict without the rows with missing values.
We fit the LinearRegression with the X and Y. I choose LinearRegression because I used it before.
Future Years is an array with the years we want to predict.
Future Predictions are just the outputs of the model that we trained.
We return future years as a one dimensional array and future predictions.

In [ ]:
def predict_future(df, column, years_to_predict=5):
    df['Year'] = df['Date'].dt.year
    X = df[['Year']].dropna()
    y = df[column].dropna()
    model = LinearRegression().fit(X, y)
    future_years = np.array([X['Year'].max() + i for i in range(1, years_to_predict + 1)]).reshape(-1, 1)
    future_predictions = model.predict(future_years)
    return future_years.flatten(), future_predictions

##Predict future values
>Run the function on the datasets.

In [ ]:
future_years_gdp, future_gdp = predict_future(gdp_df, 'GDP')
future_years_unemployment, future_unemployment = predict_future(unemployment_df, 'Youth Unemployment')
future_years_inflation, future_inflation = predict_future(inflation_df, 'Inflation')
future_years_stock_market, future_stock_market = predict_future(stock_market_df, 'Stock Market')

##Create a figure with subplots

In [ ]:
fig = make_subplots(rows=4, cols=2, subplot_titles=("GDP Growth", "GDP Growth % Change", "Inflation", "Inflation % Change",
                                                    "Stock Market", "Stock Market % Change", "Youth Unemployment", "Youth Unemployment % Change"))

##Original data plots

In [ ]:
fig.add_trace(go.Scatter(x=gdp_df['Date'], y=gdp_df['GDP'], mode='lines', name='GDP Growth'), row=1, col=1)
fig.add_trace(go.Scatter(x=inflation_df['Date'], y=inflation_df['Inflation'], mode='lines', name='Inflation'), row=2, col=1)
fig.add_trace(go.Scatter(x=stock_market_df['Date'], y=stock_market_df['Stock Market'], mode='lines', name='Stock Market'), row=3, col=1)
fig.add_trace(go.Scatter(x=unemployment_df['Date'], y=unemployment_df['Youth Unemployment'], mode='lines', name='Youth Unemployment'), row=4, col=1)

##Percentage change plots



In [ ]:
fig.add_trace(go.Bar(x=gdp_percent_change['Year'], y=gdp_percent_change['Percentage Change'], name='% GDP Growth'), row=1, col=2)
fig.add_trace(go.Bar(x=inflation_percent_change['Year'], y=inflation_percent_change['Percentage Change'], name='% Inflation Change'), row=2, col=2)
fig.add_trace(go.Bar(x=stock_market_percent_change['Year'], y=stock_market_percent_change['Percentage Change'], name='% Stock Market Change'), row=3, col=2)
fig.add_trace(go.Bar(x=unemployment_percent_change['Year'], y=unemployment_percent_change['Percentage Change'], name='% Youth Unemployment Change'), row=4, col=2)

##Future predictions plots

In [ ]:
fig.add_trace(go.Scatter(x=future_years_gdp, y=future_gdp, mode='lines+markers', name='Predicted Future GDP Growth'), row=1, col=1)
fig.add_trace(go.Scatter(x=future_years_unemployment, y=future_unemployment, mode='lines+markers', name='Predicted Future Youth Unemployment'), row=4, col=1)
fig.add_trace(go.Scatter(x=future_years_inflation, y=future_inflation, mode='lines+markers', name='Predicted Future Inflation'), row=2, col=1)
fig.add_trace(go.Scatter(x=future_years_stock_market, y=future_stock_market, mode='lines+markers', name='Predicted Future Stock Market'), row=3, col=1)

##Update layout

In [ ]:
fig.update_layout(height=1200, width=1200)

##Show the figure

In [ ]:
fig.show()